In [1]:
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from nltk import word_tokenize

MIN_LENGTH = 4
MAX_LENGTH = 15
START, START_IDX = '<s>',  0
END, END_IDX = '</s>', 1
UNK, UNK_IDX = 'UNK', 2

SOS_token = START_IDX
EOS_token = END_IDX

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()

# Lets load our dictionaries.
f_eng = open('./vocabs/simple_english_vocab.Dictionary.pkl', 'rb')
english_vocab = pickle.load(f_eng)

f_ind = open('./vocabs/simple_indo_vocab.Dictionary.pkl', 'rb')
indo_vocab = pickle.load(f_ind)

In [2]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [3]:
def vectorize_sent(sent, vocab):
    return vocab.doc2idx([START] + word_tokenize(sent.lower()) + [END], unknown_word_index=2)

def variable_from_sent(sent, vocab):
    vsent = vectorize_sent(sent, vocab)
    #print(vsent)
    result = Variable(torch.LongTensor(vsent).view(-1, 1))
    #print(result)
    return result.cuda() if use_cuda else result

In [4]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # input_tensor = tensorFromSentence(input_lang, sentence)
        input_tensor = variable_from_sent(sentence, english_vocab)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('</s>')
                break
            else:
                decoded_words.append(indo_vocab.id2token[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [5]:
hidden_size = 256
encoder = EncoderRNN(len(english_vocab), hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, len(indo_vocab), dropout_p=0.1).to(device)

### Load a network weight snapshot

In [6]:
ENCODER_PATH = 'results/encoder-{}.pth'
DECODER_PATH = 'results/decoder-{}.pth'

EPOCH_NO = 70000

encoder.load_state_dict(torch.load(ENCODER_PATH.format(EPOCH_NO),map_location=device))
encoder.eval()

EncoderRNN(
  (embedding): Embedding(1249, 256)
  (gru): GRU(256, 256)
)

In [7]:
attn_decoder.load_state_dict(torch.load(DECODER_PATH.format(EPOCH_NO),map_location=device))
attn_decoder.eval()

AttnDecoderRNN(
  (embedding): Embedding(1363, 256)
  (attn): Linear(in_features=512, out_features=15, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=1363, bias=True)
)

In [8]:
input_sentence = "Tom is a good man"
output_words, attentions = evaluate(encoder, attn_decoder, input_sentence)
output_sentence = ' '.join(output_words)
print('>', input_sentence)
print('<', output_sentence)

> Tom is a good man
< <s> tom adalah pria yang baik . </s>


### Load an array of network weights snapshots

In [27]:
test_id = english_vocab.doc2idx(["mofo"])
print(test_id)

[-1]


In [49]:
ENCODER_PATH = 'results/encoder-{}.pth'
DECODER_PATH = 'results/decoder-{}.pth'

EPOCH_NO = [1000, 2000, 3000, 4000, 8000, 10000, 20000, 40000, 70000]

def load_weights(encoder, decoder, epoch_no, device):
    encoder.load_state_dict(torch.load(ENCODER_PATH.format(epoch_no),map_location=device))
    decoder.load_state_dict(torch.load(DECODER_PATH.format(epoch_no),map_location=device))
    return encoder, decoder


def input_validation(input_text, english_vocab):
    max_words_required = MAX_LENGTH - 2
    min_words_required = MIN_LENGTH
    input_tokenized = word_tokenize(input_text.lower())
    final_text = None
    message = ""
    if not min_words_required <= len(input_tokenized) <= max_words_required:
        message = "The input sentence should be between {} and {} words".format(MIN_LENGTH, MAX_LENGTH - 2)
    else:
        input_ids = english_vocab.doc2idx(input_tokenized)
        unknown_tokens = []
        for key, val in enumerate(input_ids):
            if val == -1:
                unknown_token = input_tokenized[key]
                unknown_tokens.append(unknown_token)
                input_tokenized[key] = UNK.lower()
                print("'{}' is not found in the english corpus".format(unknown_token))
        final_text = " ".join(input_tokenized)
    return final_text, message

input_sentence = "I am so tall"
final_text, message = input_validation(input_sentence, english_vocab)

if final_text:
    print('>', input_sentence)
    print('>>', final_text)

    for epoch in EPOCH_NO:
        encoder, decoder = load_weights(encoder, attn_decoder, epoch, device)
        output_words, attentions = evaluate(encoder, decoder, final_text)
        output_sentence = ' '.join(output_words).replace('<s>', '').replace('</s>', '')
        print("Epoch:", epoch, '<', output_sentence)

> I am so tall
>> i am so tall
Epoch: 1000 <  tom di di . 
Epoch: 2000 <  aku sangat ? 
Epoch: 3000 <  aku lapar . 
Epoch: 4000 <  aku bukan siswa . 
Epoch: 8000 <  saya lapar . 
Epoch: 10000 <  saya bukan . 
Epoch: 20000 <  saya bosan . 
Epoch: 40000 <  saya bosan . 
Epoch: 70000 <  saya bosan . 
